In [ ]:
library(tidyverse)

In [2]:
library(ggplot2)
library(ggpubr)

In [3]:
library(pheatmap)

In [4]:
load('../02.differential/Deseq2_TvsN_FC2.Rdata')

In [5]:
cytokines <- read.table('cytokine.txt',header=F)

In [15]:
alldiffgenes <- unique(c(rlt_high_tn$diff %>% rownames(), rlt_low_tn$diff %>% rownames() ,rlt_neg_tn$diff %>% rownames()))

In [16]:
genes_intersect <- intersect(cytokines$V1,alldiffgenes)

In [17]:
openxlsx::write.xlsx( as.data.frame(genes_intersect),
           rowNames = FALSE, file = "cytokines.xlsx")

In [22]:
rlt_high_tn$all[genes_intersect,] %>% select(log2FoldChange,padj) %>% 
    mutate(log2FoldChange=ifelse(padj<0.05,log2FoldChange,0)) %>% 
    mutate(log2FoldChange=ifelse(abs(log2FoldChange)>=1,log2FoldChange,0))  %>% 
    select(log2FoldChange) -> high_cytokines

In [23]:
rlt_low_tn$all[genes_intersect,] %>% select(log2FoldChange,padj) %>% 
    mutate(log2FoldChange=ifelse(padj<0.05,log2FoldChange,0)) %>% 
    mutate(log2FoldChange=ifelse(abs(log2FoldChange)>=1,log2FoldChange,0))  %>% 
    select(log2FoldChange) -> low_cytokines

In [24]:
rlt_neg_tn$all[genes_intersect,] %>% select(log2FoldChange,padj) %>% 
    mutate(log2FoldChange=ifelse(padj<0.05,log2FoldChange,0)) %>% 
    mutate(log2FoldChange=ifelse(abs(log2FoldChange)>=1,log2FoldChange,0))  %>% 
    select(log2FoldChange) -> neg_cytokines

In [27]:
merge_cytokines_raw <- cbind(high_cytokines,low_cytokines,neg_cytokines)

In [28]:
merge_cytokines <- cbind(high_cytokines,low_cytokines,neg_cytokines)

In [29]:
colnames(merge_cytokines) <- c("high","low","neg")

In [30]:
merge_cytokines[merge_cytokines>0]=1
merge_cytokines[merge_cytokines<0]=-1

In [31]:
temp <- hclust(dist(merge_cytokines))

In [32]:
merge_cytokines <- merge_cytokines[temp$order,]

In [35]:
#openxlsx::write.xlsx(merge_cytokines, rowNames = TRUE, file = "cytokines_order.xlsx")

In [43]:
merge_cytokines_reorder <- openxlsx::read.xlsx(xlsxFile = "cytokines_order.xlsx",rowNames = TRUE)

In [44]:
while (!is.null(dev.list())) dev.off()

In [45]:
options(repr.plot.width=3,repr.plot.height=10)
pheatmap(merge_cytokines_reorder,cluster_cols = F,cluster_rows = F,color=c("#008110","white", "#BF3730"),
         filename = "cytokines.pdf",width = 3,height=10,
        fontsize_row = 5) 


In [46]:
merge_cytokines_raw[rownames(merge_cytokines_reorder),] -> merge_cytokines_raw2

In [47]:
merge_cytokines_raw2[merge_cytokines_raw2>5]=5

In [48]:
merge_cytokines_raw2[merge_cytokines_raw2< -5]=-5

In [49]:
colnames(merge_cytokines_raw2) <- c("high","low","neg")

In [51]:
options(repr.plot.width=3,repr.plot.height=10)
pheatmap(merge_cytokines_raw2,cluster_cols = F,cluster_rows = F, color=colorRampPalette(c("#008110","white", "#BF3730"))(100),
                filename = "cytokines2.pdf",width = 3,height=10,
         fontsize_row = 5)
